<a href="https://colab.research.google.com/github/YOUSUBEEN/python_colab/blob/main/API_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 서울시 부동산 XML API 예제


In [1]:
# 라이브러리 가져오기

import lxml
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import math

In [2]:
# 인증키 가져오기
# 부동산 실거래가

service_key = ""  # 어디든 공개X
url = f'http://openapi.seoul.go.kr:8088/{service_key}/xml/tbLnOpendataRtmsV/1/5/'
print(url)

http://openapi.seoul.go.kr:8088/6e445a585a746e713431694c625553/xml/tbLnOpendataRtmsV/1/5/


In [3]:
req = requests.get(url)
req.content

b'<?xml version="1.0" encoding="UTF-8"?>\n<tbLnOpendataRtmsV>\n<list_total_count>2666181</list_total_count>\n<RESULT>\n<CODE>INFO-000</CODE>\n<MESSAGE>\xec\xa0\x95\xec\x83\x81 \xec\xb2\x98\xeb\xa6\xac\xeb\x90\x98\xec\x97\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4</MESSAGE>\n</RESULT>\n<row>\n<ACC_YEAR>2023</ACC_YEAR>\n<SGG_CD>11380</SGG_CD>\n<SGG_NM>\xec\x9d\x80\xed\x8f\x89\xea\xb5\xac</SGG_NM>\n<BJDONG_CD>10500</BJDONG_CD>\n<BJDONG_NM>\xea\xb5\xac\xec\x82\xb0\xeb\x8f\x99</BJDONG_NM>\n<LAND_GBN>1</LAND_GBN>\n<LAND_GBN_NM>\xeb\x8c\x80\xec\xa7\x80</LAND_GBN_NM>\n<BONBEON>0162</BONBEON>\n<BUBEON>0001</BUBEON>\n<BLDG_NM>\xea\xb2\xbd\xed\x96\xa5\xed\x8c\x8c\xed\x81\xac</BLDG_NM>\n<DEAL_YMD>20230616</DEAL_YMD>\n<OBJ_AMT>60000</OBJ_AMT>\n<BLDG_AREA>124.8</BLDG_AREA>\n<TOT_AREA>0.000000</TOT_AREA>\n<FLOOR>3</FLOOR>\n<RIGHT_GBN/>\n<CNTL_YMD/>\n<BUILD_YEAR>1989</BUILD_YEAR>\n<HOUSE_TYPE>\xec\x95\x84\xed\x8c\x8c\xed\x8a\xb8</HOUSE_TYPE>\n<REQ_GBN>\xec\xa4\x91\xea\xb0\x9c\xea\xb1\xb0\xeb\x9e\x98</REQ

In [4]:
soup = BeautifulSoup(req.content, "lxml") # XML 파싱
print(soup)

<?xml version="1.0" encoding="UTF-8"?><html><body><tblnopendatartmsv>
<list_total_count>2666181</list_total_count>
<result>
<code>INFO-000</code>
<message>정상 처리되었습니다</message>
</result>
<row>
<acc_year>2023</acc_year>
<sgg_cd>11380</sgg_cd>
<sgg_nm>은평구</sgg_nm>
<bjdong_cd>10500</bjdong_cd>
<bjdong_nm>구산동</bjdong_nm>
<land_gbn>1</land_gbn>
<land_gbn_nm>대지</land_gbn_nm>
<bonbeon>0162</bonbeon>
<bubeon>0001</bubeon>
<bldg_nm>경향파크</bldg_nm>
<deal_ymd>20230616</deal_ymd>
<obj_amt>60000</obj_amt>
<bldg_area>124.8</bldg_area>
<tot_area>0.000000</tot_area>
<floor>3</floor>
<right_gbn></right_gbn>
<cntl_ymd></cntl_ymd>
<build_year>1989</build_year>
<house_type>아파트</house_type>
<req_gbn>중개거래</req_gbn>
<rdealer_lawdnm>서울 은평구</rdealer_lawdnm>
</row>
<row>
<acc_year>2023</acc_year>
<sgg_cd>11560</sgg_cd>
<sgg_nm>영등포구</sgg_nm>
<bjdong_cd>11500</bjdong_cd>
<bjdong_nm>당산동5가</bjdong_nm>
<land_gbn>1</land_gbn>
<land_gbn_nm>대지</land_gbn_nm>
<bonbeon>0009</bonbeon>
<bubeon>0003</bubeon>
<bldg_nm>당산역해링턴타워<

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


# 데이터 가공
- XML 형태를 pandas DataFrame으로 변환

In [5]:
years            = soup.find_all('acc_year')         # 접수년월
sgg_cds          = soup.find_all('sgg_cd')           # 자치구코드
sgg_nms          = soup.find_all('sgg_nm')           # 자치구명
bjdong_cds       = soup.find_all('bjdong_cd')        # 법정동코드
bjdong_nms       = soup.find_all('bjdong_nm')        # 법정동명
land_gbns        = soup.find_all('land_gbn')         # 지번구분
land_gbn_nms     = soup.find_all('land_gbn_nm')      # 지번구분명
land_gbn_nms     = soup.find_all('land_gbn_nm')      # 지번구분명
bonbeons         = soup.find_all('bonbeon')          # 본번
bubeons          = soup.find_all('bubeon')           # 부번
bldg_nms         = soup.find_all('bldg_nm')          # 건물명
deal_ymds        = soup.find_all('deal_ymd')         # 계약일
obj_amts         = soup.find_all('obj_amt')          # 물건금액(만원)
bldg_areas       = soup.find_all('bldg_area')        # 건물면적(㎡)
tot_areas        = soup.find_all('tot_area')         # 토지면적(㎡)
floors           = soup.find_all('floor')            # 층
right_gbns       = soup.find_all('right_gbn')        # 권리구분
cntl_ymds        = soup.find_all('cntl_ymd')         # 취소일
build_years      = soup.find_all('build_years')      # 건축년도
house_types      = soup.find_all('house_type')       # 건물용도
req_gbn          = soup.find_all('req_gbn')          # 신고구분
rdealer_lawdnms  = soup.find_all('rdealer_lawdnm')   # 신고한 개업공인중개사 시군구명

In [6]:
rdealer_lawdnms

[<rdealer_lawdnm>서울 은평구</rdealer_lawdnm>,
 <rdealer_lawdnm>서울 영등포구</rdealer_lawdnm>,
 <rdealer_lawdnm>서울 양천구</rdealer_lawdnm>,
 <rdealer_lawdnm>서울 송파구</rdealer_lawdnm>,
 <rdealer_lawdnm>서울 광진구</rdealer_lawdnm>]

In [7]:
# 반복문 활용

year_list           = []
sgg_cd_list         = []
bldg_nm_list        = []
obj_amt_list        = []
house_type_list     = []
rdealer_lawdnm_list = []

for year, sgg_cd, bldg_nm, obj_amt, house_type, rdealer_lawdnm in zip(years, sgg_cds, bldg_nms, obj_amts, house_types, rdealer_lawdnms):
  year_list.append(year.get_text())
  sgg_cd_list.append(sgg_cd.get_text())
  bldg_nm_list.append(bldg_nm.get_text())
  obj_amt_list.append(obj_amt.get_text())
  house_type_list.append(house_type.get_text())
  rdealer_lawdnm_list.append(rdealer_lawdnm.get_text())

df = pd.DataFrame({
    "acc_year": year_list,
    "sgg_cd": sgg_cd_list,
    "bldg_nm" : bldg_nm_list,
    "obj_amt": obj_amt_list,
    "house_type" : house_type_list,
    "rdealer_lawdnm": rdealer_lawdnm_list
})

df

,acc_year,sgg_cd,bldg_nm,obj_amt,house_type,rdealer_lawdnm
0,2023,11380,경향파크,60000,아파트,서울 은평구
1,2023,11560,당산역해링턴타워,22750,오피스텔,서울 영등포구
2,2023,11500,수아트빌,23500,연립다세대,서울 양천구
3,2023,11710,한양빌리지,22000,연립다세대,서울 송파구
4,2023,11215,환성빌,30500,연립다세대,서울 광진구


## JSON 형태로하기

In [8]:
# 인증키 가져오기

service_key = ""  # 어디든 공개X
url = f'http://openapi.seoul.go.kr:8088/{service_key}/json/tbLnOpendataRtmsV/1/5/'
print(url)

http://openapi.seoul.go.kr:8088/6e445a585a746e713431694c625553/json/tbLnOpendataRtmsV/1/5/


In [9]:
req = requests.get(url)
content = req.json()
content

{'tbLnOpendataRtmsV': {'list_total_count': 2666181,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'ACC_YEAR': '2023',
    'SGG_CD': '11380',
    'SGG_NM': '은평구',
    'BJDONG_CD': '10500',
    'BJDONG_NM': '구산동',
    'LAND_GBN': '1',
    'LAND_GBN_NM': '대지',
    'BONBEON': '0162',
    'BUBEON': '0001',
    'BLDG_NM': '경향파크',
    'DEAL_YMD': '20230616',
    'OBJ_AMT': '60000',
    'BLDG_AREA': 124.8,
    'TOT_AREA': 0.0,
    'FLOOR': 3.0,
    'RIGHT_GBN': '',
    'CNTL_YMD': '',
    'BUILD_YEAR': '1989',
    'HOUSE_TYPE': '아파트',
    'REQ_GBN': '중개거래',
    'RDEALER_LAWDNM': '서울 은평구'},
   {'ACC_YEAR': '2023',
    'SGG_CD': '11560',
    'SGG_NM': '영등포구',
    'BJDONG_CD': '11500',
    'BJDONG_NM': '당산동5가',
    'LAND_GBN': '1',
    'LAND_GBN_NM': '대지',
    'BONBEON': '0009',
    'BUBEON': '0003',
    'BLDG_NM': '당산역해링턴타워',
    'DEAL_YMD': '20230616',
    'OBJ_AMT': '22750',
    'BLDG_AREA': 23.92,
    'TOT_AREA': 33.23,
    'FLOOR': 18.0,
    'RIGHT_GBN': '',
    'CNTL

In [10]:
content["tbLnOpendataRtmsV"].keys()

dict_keys(['list_total_count', 'RESULT', 'row'])

In [11]:
# json을 데이터프레임으로 변환

pd.DataFrame(content['tbLnOpendataRtmsV']['row'])

,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BONBEON,BUBEON,BLDG_NM,...,OBJ_AMT,BLDG_AREA,TOT_AREA,FLOOR,RIGHT_GBN,CNTL_YMD,BUILD_YEAR,HOUSE_TYPE,REQ_GBN,RDEALER_LAWDNM
0,2023,11380,은평구,10500,구산동,1,대지,0162,0001,경향파크,...,60000,124.80,0.00,3.0,,,1989,아파트,중개거래,서울 은평구
1,2023,11560,영등포구,11500,당산동5가,1,대지,0009,0003,당산역해링턴타워,...,22750,23.92,33.23,18.0,,,2015,오피스텔,중개거래,서울 영등포구
2,2023,11500,강서구,10300,화곡동,1,대지,0355,0037,수아트빌,...,23500,39.58,23.63,5.0,,,2015,연립다세대,중개거래,서울 양천구
3,2023,11710,송파구,10400,송파동,1,대지,0010,0000,한양빌리지,...,22000,40.77,17.94,4.0,,,1992,연립다세대,중개거래,서울 송파구
4,2023,11215,광진구,10700,화양동,1,대지,0036,0076,환성빌,...,30500,25.43,23.21,4.0,,,2016,연립다세대,중개거래,서울 광진구
